These are some **internee-level** questions that are still **brain-twisters** — questions that *look approachable* but will still light a cerebral fire under anyone who’s too comfy.
These are designed to test logical depth, DP instinct, and data structure creativity — **without advanced prerequisites** like heavy math or domain-specific theory.

---

💭 **Note:** None of these need tree-heavy math, heavy libraries, or weird edge cases.
But they all need **mental dexterity**.

### 🔀 **1. The ASCII Duel: Interwoven Greed**

**Problem:**  
You are given two strings `A` and `B`. You must interleave them (keeping relative order from both intact) to form a new string `C`, such that the **ASCII sum of `C` is maximized**.

At each step, you choose whether to take the next char from `A` or `B`.

#### 🔍 Test Cases:
```python
Input:
A = "ace"
B = "bd"
Output:  587  # "abcde" has max ASCII sum of 97+98+99+100+101 = 495? Nah. Best is "abce" = 97+98+99+101 = 395? Try better.

Input:
A = "gk"
B = "a"
Output:  208  # "gak" = 103+97+107, best order
```

#### 💡 Hint:
- Use **DP** with `dp[i][j] = max ASCII sum from A[i:] and B[j:]`
- It’s like **merge** in merge sort, but **greedy with a brain** (ASCII-max instead of alphabetic order)
- Keep memo table to avoid recomputation

In [1]:
def ascii_duel(a: str, b: str):
    # Step 1: Create a map of ASCII values for all unique characters
    ascii_values = {char: ord(char) for char in set(a + b)}

    # Step 2: Get original indexed characters from both strings
    indexed_a = list(enumerate(a))
    indexed_b = list(enumerate(b))

    # Step 3: Filter characters that are within the ASCII max-min range
    min_ascii = min(ascii_values.values())
    max_ascii = max(ascii_values.values())

    filtered_a = [char for idx, char in indexed_a if min_ascii < ascii_values[char] <= max_ascii]
    filtered_b = [char for idx, char in indexed_b if min_ascii < ascii_values[char] <= max_ascii]

    # Step 4: Interleave based on highest ASCII value, maintaining order
    i = j = 0
    result = []

    while i < len(filtered_a) or j < len(filtered_b):
        char_a = filtered_a[i] if i < len(filtered_a) else ''
        char_b = filtered_b[j] if j < len(filtered_b) else ''

        if char_a and (not char_b or ascii_values[char_a] >= ascii_values[char_b]):
            result.append(char_a)
            i += 1
        elif char_b:
            result.append(char_b)
            j += 1

    final_string = ''.join(result)
    ascii_sum = sum(ord(c) for c in final_string)

    return final_string, ascii_sum

res, val = ascii_duel("facade", "acid")
print(res)
print(val)

fcdecid
706


### 🧬 **2. Quantum Grid Lite**

**Problem:**  
Given a grid `G` of 0s and 1s, where 1 is a **quantum gate** (that flips the current cell value), move from `(0, 0)` to `(m-1, n-1)` using only **right or down**.  
Quantum gates only affect the cell you're *standing in*.  
Count minimum steps or return -1 if no path.

#### 🔍 Test Cases:
```python
Input:
G = [[0, 1, 0],
     [1, 0, 1],
     [0, 1, 0]]
Output:  4

Input:
G = [[0, 1],
     [1, 1]]
Output:  2

Input:
G = [[1, 1],
     [1, 0]]
Output:  -1
```

#### 💡 Hint:
- Use **BFS** with tracking of `(i, j, flipped)` as state.
- A cell may be **visited twice** — once with flip, once without
- Don’t assume the value in the matrix is fixed — it’s **contextual per path**

In [2]:
def count_paths(grid):
    rows, cols = len(grid), len(grid[0])
    dp = {}

    for i in range(rows):
        for j in range(cols):
            if grid[i][j] == 1:
                continue  # Blocked cell, no path goes through
            if i == 0 and j == 0:
                dp[(0, 0)] = 1  # Starting point
            else:
                from_top = dp.get((i - 1, j), 0)
                from_left = dp.get((i, j - 1), 0)
                dp[(i, j)] = from_top + from_left

    return dp.get((rows - 1, cols - 1), 0)

grid = [
    [0, 0, 0],
    [0, 1, 0],
    [0, 0, 0]
]
print(count_paths(grid))

2


### 🌀 **3. Fractal Growth Simulator**

**Problem:**  
You’re given a string and **rules** like `('A', 'AB')`. Apply rules **n times** recursively.

Compute only the **length** of the final string after n iterations, **not the string itself**.

#### 🔍 Test Cases:
```python
Input:
S = "A"
Rules = [('A', 'AB'), ('B', 'A')]
n = 2
Output: 5

Input:
S = "AB"
Rules = [('A', 'B'), ('B', 'AB')]
n = 3
Output: 8
```

#### 💡 Hint:
- Think **DP on (char, depth)** → Cache length for each `(char, n)`
- At each level, replace `char` by rule or 1 if no rule
- Memoization is your fuel cell here

In [3]:
from collections import defaultdict

def fractal_growth_simulator(s: str, rules: list[tuple[str, str]], n: int):
    rules_map = {k : v for k, v in rules}
    memo = {}

    def expand(char, depth):
        if (char, depth) in memo:
            return memo[(char, depth)]
        if depth == 0:
            return {char: 1}
        if char not in rules_map:
            return {char :1}

        result = defaultdict(int)
        for c in rules_map[char]:
            expanded = expand(char = c, depth=depth-1)
            for k, v in expanded.items():
                result[k] += v

        memo[(char, depth)] = result
        return result

    total_counts = defaultdict(int)
    for c in s:
        expanded = expand(c, n)
        for k, v in expanded.items():
            total_counts[k] += v

    return sum(total_counts.values())

print(fractal_growth_simulator("A", [('A', 'AB'), ('B', 'A')], 2))
print(fractal_growth_simulator("AB", [('A', 'B'), ('B', 'AB')], 3))
print(fractal_growth_simulator("A", [('A', 'AB'), ('B', 'A')], 20))

3
8
17711


### 💰 **4. Budget Bender: Mini Knapsack**

**Problem:**  
You’re given `n` tasks. Each has:
- cost (resource needed)
- profit (reward for doing task)

You have total resource `R`.  
Select subset of tasks with max profit, total cost ≤ R.

#### 🔍 Test Cases:
```python
Input:
Tasks = [(2, 4), (3, 5), (1, 3)]
R = 4
Output: 7  # (1,3) + (3,5) is too much; best is (1,3)+(2,4)

Input:
Tasks = [(5, 10), (4, 40), (6, 30), (3, 50)]
R = 10
Output: 90
```

#### 💡 Hint:
- Classic **0/1 Knapsack** — but code it **yourself**, no libraries
- `dp[i][r] = max profit using first i tasks with r resources`
- You’ll love how beautifully brute-force evolves into elegance here

In [4]:
def budget_bender(tasks: list[tuple[int, int]], r: int) -> int:
    dp = [0] * (r + 1)

    for cost, profit in tasks:
        print("Cost: ", cost)
        print("Profit: ", profit)
        for i in range(r, cost - 1, -1):
            dp[i] = max(dp[i], dp[i - cost] + profit)
            print(f"dp at {i}", dp[i])

    return dp[r]

tasks = [(5, 10), (4, 7), (6, 15), (3, 5)]
r = 10
print("Final budget: ", budget_bender(tasks, r))

Cost:  5
Profit:  10
dp at 10 10
dp at 9 10
dp at 8 10
dp at 7 10
dp at 6 10
dp at 5 10
Cost:  4
Profit:  7
dp at 10 17
dp at 9 17
dp at 8 10
dp at 7 10
dp at 6 10
dp at 5 10
dp at 4 7
Cost:  6
Profit:  15
dp at 10 22
dp at 9 17
dp at 8 15
dp at 7 15
dp at 6 15
Cost:  3
Profit:  5
dp at 10 22
dp at 9 20
dp at 8 15
dp at 7 15
dp at 6 15
dp at 5 10
dp at 4 7
dp at 3 5
Final budget:  22


### 🌐 **5. Network Ping Simulator**

**Problem:**  
You're given a graph `G` with `n` nodes and `edges = (u, v, cost)`.

Every second `t`, the cost of an edge becomes:  
`cost + t % 3`  
You need to process `Q` queries of form `(start, end, t)` and return **shortest path** from `start` to `end` at time `t`.

#### 🔍 Test Cases:
```python
Input:
edges = [(0,1,1), (1,2,2), (0,2,4)]
queries = [(0, 2, 1), (0, 2, 3)]
Output: [4, 5]
```

#### 💡 Hint:
- For each query, build weights `cost + (t % 3)`
- Use **Dijkstra’s Algorithm** with fresh edge weights per query
- If doing many queries → Preprocess edges with time offset mod 3 (lazy caching?)

In [5]:
graph = {
    0: {
        1: 1,
        2: 4,
        3: 9,
        4: 16,
        5: 25,
        6: 36,
        7: 49
    },
    1: {
        0: 1,
        2: 5,
        3: 10,
        4: 17,
        5: 26,
        6: 37,
        7: 50
    },
    2: {
        0: 2,
        1: 3,
        3: 11,
        4: 18,
        5: 27,
        6: 38,
        7: 51
    },
    3: {
        0: 3,
        1: 4,
        2: 7,
        4: 19,
        5: 28,
        6: 39,
        7: 52
    },
    4: {
        0: 4,
        1: 5,
        2: 8,
        3: 13,
        5: 29,
        6: 40,
        7: 53
    },
    5: {
        0: 5,
        1: 6,
        2: 9,
        3: 14,
        4: 21,
        6: 41,
        7: 54
    },
    6: {
        0: 6,
        1: 7,
        2: 10,
        3: 15,
        4: 20,
        5: 31,
        7: 55
    },
    7: {
        0: 7,
        1: 8,
        2: 11,
        3: 16,
        4: 23,
        5: 32,
        6: 43
    }
}

graph

{0: {1: 1, 2: 4, 3: 9, 4: 16, 5: 25, 6: 36, 7: 49},
 1: {0: 1, 2: 5, 3: 10, 4: 17, 5: 26, 6: 37, 7: 50},
 2: {0: 2, 1: 3, 3: 11, 4: 18, 5: 27, 6: 38, 7: 51},
 3: {0: 3, 1: 4, 2: 7, 4: 19, 5: 28, 6: 39, 7: 52},
 4: {0: 4, 1: 5, 2: 8, 3: 13, 5: 29, 6: 40, 7: 53},
 5: {0: 5, 1: 6, 2: 9, 3: 14, 4: 21, 6: 41, 7: 54},
 6: {0: 6, 1: 7, 2: 10, 3: 15, 4: 20, 5: 31, 7: 55},
 7: {0: 7, 1: 8, 2: 11, 3: 16, 4: 23, 5: 32, 6: 43}}

In [6]:
def make_symmetric_graph(graph: dict[int, dict[int, int]]) -> dict[int, dict[int, int]]:
    symmetric_graph = {}

    for u in graph:
        for v, w in graph[u].items():
            # Initialize if the node doesn't exist yet
            if u not in symmetric_graph:
                symmetric_graph[u] = {}
            if v not in symmetric_graph:
                symmetric_graph[v] = {}

            # Set weight in both directions
            symmetric_graph[u][v] = w
            symmetric_graph[v][u] = w  # Make sure it's mirrored

    return symmetric_graph

symmetric_graph = make_symmetric_graph(graph)
symmetric_graph

{0: {1: 1, 2: 2, 3: 3, 4: 4, 5: 5, 6: 6, 7: 7},
 1: {0: 1, 2: 3, 3: 4, 4: 5, 5: 6, 6: 7, 7: 8},
 2: {0: 2, 1: 3, 3: 7, 4: 8, 5: 9, 6: 10, 7: 11},
 3: {0: 3, 1: 4, 2: 7, 4: 13, 5: 14, 6: 15, 7: 16},
 4: {0: 4, 1: 5, 2: 8, 3: 13, 5: 21, 6: 20, 7: 23},
 5: {0: 5, 1: 6, 2: 9, 3: 14, 4: 21, 6: 31, 7: 32},
 6: {0: 6, 1: 7, 2: 10, 3: 15, 4: 20, 5: 31, 7: 43},
 7: {0: 7, 1: 8, 2: 11, 3: 16, 4: 23, 5: 32, 6: 43}}

**Note**: I'd prefer to use `graph` instead of `symmetric_graph`; because, `graph` has no loops.

In [7]:
edges = []
for u in graph:
    for vertex, weight in graph[u].items():
        edges.append((u, vertex, weight))
edges

[(0, 1, 1),
 (0, 2, 4),
 (0, 3, 9),
 (0, 4, 16),
 (0, 5, 25),
 (0, 6, 36),
 (0, 7, 49),
 (1, 0, 1),
 (1, 2, 5),
 (1, 3, 10),
 (1, 4, 17),
 (1, 5, 26),
 (1, 6, 37),
 (1, 7, 50),
 (2, 0, 2),
 (2, 1, 3),
 (2, 3, 11),
 (2, 4, 18),
 (2, 5, 27),
 (2, 6, 38),
 (2, 7, 51),
 (3, 0, 3),
 (3, 1, 4),
 (3, 2, 7),
 (3, 4, 19),
 (3, 5, 28),
 (3, 6, 39),
 (3, 7, 52),
 (4, 0, 4),
 (4, 1, 5),
 (4, 2, 8),
 (4, 3, 13),
 (4, 5, 29),
 (4, 6, 40),
 (4, 7, 53),
 (5, 0, 5),
 (5, 1, 6),
 (5, 2, 9),
 (5, 3, 14),
 (5, 4, 21),
 (5, 6, 41),
 (5, 7, 54),
 (6, 0, 6),
 (6, 1, 7),
 (6, 2, 10),
 (6, 3, 15),
 (6, 4, 20),
 (6, 5, 31),
 (6, 7, 55),
 (7, 0, 7),
 (7, 1, 8),
 (7, 2, 11),
 (7, 3, 16),
 (7, 4, 23),
 (7, 5, 32),
 (7, 6, 43)]

In [8]:
def is_directed_graph(graph):
    """
    Checks if a graph represented by an adjacency matrix is directed.

    Args:
        graph: A list of lists representing the adjacency matrix of the graph.

    Returns:
        True if the graph is directed, False otherwise.
    """
    num_vertices = len(graph)
    for i in range(num_vertices):
        for j in range(num_vertices):
            if graph[i][j] != graph[j][i]:
                return True  # Found an edge where the direction matters
    return False  # No asymmetric edges found, it's an undirected graph

# Example Usage
graph1 = [
    [0, 1, 0],
    [0, 0, 1],
    [0, 0, 0]
]
print(f"Graph 1 is directed: {is_directed_graph(graph1)}")

graph2 = [
    [0, 1, 0],
    [1, 0, 1],
    [0, 1, 0]
]
print(f"Graph 2 is directed: {is_directed_graph(graph2)}")

Graph 1 is directed: True
Graph 2 is directed: False


In my 1st approach, I'd use a custom provided `edges`, representing a graph and a `query` containing tuples of start, end and time, to find the shortest path. I'll be using bellman-ford because the custom `edges` may have negative weights or even cycles in it. I've optimized it to stop when `end` is reached.

In [9]:
def ping_simulator(
    edges: list[tuple[object, object, int]],
    time_ticks: list[int],
    start: object,
    end: object
) -> dict:
    dist = {}
    for u, v, _ in edges:
        dist[u] = float('inf')
        dist[v] = float('inf')

    dist[start] = 0
    shortest_path = {}

    required_edges = []
    for u, v, w in edges:
        if (u == end or v == end) or (u == start or v == start):
            required_edges.append((u, v, w))

    for time_tick in time_ticks:
        updated = False  # Track whether we updated the `end` node

        for u, v, weight in required_edges:
            ping_cost = weight + (time_tick % 3)

            if dist[u] + ping_cost < dist[v]:
                dist[v] = dist[u] + ping_cost
                shortest_path[v] = {
                    'from': u,
                    'ping cost': ping_cost
                }
                if v == end:
                    updated = True

            if dist[v] + ping_cost < dist[u]:
                dist[u] = dist[v] + ping_cost
                shortest_path[u] = {
                    'from': v,
                    'ping cost': ping_cost
                }
                if u == end:
                    updated = True

        # If we've found and finalized a path to `end`, we can stop early
        if dist[end] != float('inf') and updated:
            break

    return shortest_path

In [10]:
graph_edges = []
start = 4
end = 1
time_ticks = [1,2,3,4]

ping_simulator(edges=edges, time_ticks=time_ticks, start=start, end=end)

{0: {'from': 4, 'ping cost': 5},
 1: {'from': 4, 'ping cost': 6},
 2: {'from': 4, 'ping cost': 9},
 3: {'from': 4, 'ping cost': 14},
 5: {'from': 1, 'ping cost': 7},
 6: {'from': 1, 'ping cost': 8},
 7: {'from': 1, 'ping cost': 9}}

### Dijkstra based solution

There's an another approach, to this problem using **dijkstra algorithm**, but implementation will be for `start` to `end` vertices' shortest path. However, this approach isn't cyclical graph optimized, and can't handle negative weights. But has time complexity less then that of the previous one.

In [11]:
import heapq
from collections import defaultdict

def build_graph(edges):
    graph = defaultdict(list)
    for u, v, cost in edges:
        graph[u].append((v, cost))
        graph[v].append((u, cost))
    return graph

def dijkstra(graph, start, end, t):
    heap = [(0, start)]
    visited = set()
    dist = defaultdict(lambda: float('inf'))
    dist[start] = 0

    while heap:
        curr_dist, u = heapq.heappop(heap)
        if u == end:
            return curr_dist
        if u in visited:
            continue
        visited.add(u)

        for v, cost in graph[u]:
            effective_cost = cost + (t % 3)
            if curr_dist + effective_cost < dist[v]:
                dist[v] = curr_dist + effective_cost
                heapq.heappush(heap, (dist[v], v))
    return -1

def network_ping_simulator(edges, queries):
    graph = build_graph(edges)
    results = []
    for start, end, t in queries:
        results.append(dijkstra(graph, start, end, t))
    return results

In [12]:
edges = [(0, 1, 1), (1, 2, 2), (0, 2, 4)]
queries = [(0, 2, 1), (0, 2, 3)]

print(network_ping_simulator(edges, queries))

[5, 3]


### 🔧 **Step-by-Step Breakdown**

### 1️⃣ `build_graph(edges)`
```python
def build_graph(edges):
    graph = defaultdict(list)
    for u, v, cost in edges:
        graph[u].append((v, cost))
        graph[v].append((u, cost))  # undirected
    return graph
```

✅ This converts a **list of edges** into a usable graph (like an adjacency list).

- Each `edge = (u, v, cost)` becomes:
  ```python
  graph[u] -> [(v, cost)]
  graph[v] -> [(u, cost)]  # since it's undirected
  ```

---

### 2️⃣ `dijkstra(graph, start, end, t)`
```python
def dijkstra(graph, start, end, t):
```

🔁 **Modified Dijkstra’s Algorithm** — used to find the **shortest path** between `start` and `end`, **at time t**.

#### ⏱️ The Catch:
```python
effective_cost = cost + (t % 3)
```
> Every edge's cost **increases** depending on the second `t` using `% 3`.

### Breakdown of the logic:
```python
heap = [(0, start)]  # Priority queue: (distance_so_far, current_node)
visited = set()
dist = defaultdict(lambda: float('inf'))
dist[start] = 0
```

- `heapq` is a **priority queue** — always gives you the lowest-cost node to explore next.
- `visited` avoids re-processing nodes.
- `dist[node]` keeps track of **minimum ping time** from `start` to that node.

#### The main loop:
```python
while heap:
    curr_dist, u = heapq.heappop(heap)
```
- Pull out the node `u` with the **lowest** distance so far.

If this is the `end` node → **return the answer** right away:
```python
if u == end:
    return curr_dist
```

Otherwise, explore its neighbors:
```python
for v, cost in graph[u]:
    effective_cost = cost + (t % 3)
    if curr_dist + effective_cost < dist[v]:
        dist[v] = curr_dist + effective_cost
        heapq.heappush(heap, (dist[v], v))
```

> This is Dijkstra's **"relaxation" step** but with the time-modified cost.

If `end` is never reached:
```python
return -1
```

---

### 3️⃣ `network_ping_simulator(edges, queries)`
```python
def network_ping_simulator(edges, queries):
    graph = build_graph(edges)
    results = []
    for start, end, t in queries:
        results.append(dijkstra(graph, start, end, t))
    return results
```

This is the **orchestrator function**:
- Build the graph
- Run Dijkstra for every query `(start, end, t)`
- Collect the results

---

### 4️⃣ Example Run:
```python
edges = [(0, 1, 1), (1, 2, 2), (0, 2, 4)]
queries = [(0, 2, 1), (0, 2, 3)]
```

#### First Query → `(0, 2, 1)`
- Edge weights get `+ (1 % 3) = +1`
  - So effective weights:
    - 0→1 = 2
    - 1→2 = 3
    - 0→2 = 5
- Best path: 0→1→2 = 2+3 = 5 (but 0→2 directly is 5 too)
✅ So, shortest is 0→2 = **5**

### ✅ TL;DR Summary

| Component               | Role                                             |
|------------------------|--------------------------------------------------|
| `build_graph`          | Converts edge list → adjacency list              |
| `dijkstra`             | Finds shortest path from `start` to `end` using dynamic cost |
| `cost + (t % 3)`       | Makes the cost change depending on time `t`      |
| `network_ping_simulator` | Handles multiple queries                        |
| `heapq`                | Ensures we always process the **shortest path** so far |